# Reader monad demo

**When**: JHU Semantics seminar, 2/13/24

**Author**: Kyle Rawlins

This notebook steps through the reader monad, applied to intensional semantics. It is not a standalone intro to monads or to the reader monad. For a primary reference, see:

* 2002. Ken Shan. [Monads for natural language semantics](https://arxiv.org/pdf/cs/0205026). In Proceedings of the 2001 European Summer School in Logic, Language and Information student session, ed. Kristina Striegnitz, 285-298.

In [ ]:
lamb.display.default(style=lamb.display.DerivStyle.PROOF)

# basic composition system with an intensional type
type_s = types.BasicType("s")
system = lang.td_system.copy()
system.add_basic_type(type_s)
lang.set_system(system)
system

**First**: define the core monad operations (including lifted apply).

In [ ]:
%%lamb
# monad operations
unit = L a_X : L w_s : a
bind = L m_<s,X> : L k_<X,<s,Y>> : L w_s : (k(m(w))(w))
# apply = L f_<s,<X,Y>> : L x_<s,X> : L w_s : f(w)(x(w))
apply = L f_<s,<X,Y>> : L x_<s,X> : bind(f)(L a_<X,Y> : bind(x_<s,X>)(L b_X : unit(a(b))))

**Second**: add composition operations to the existing system using these monad operations. We keep FA.

In [ ]:
system.add_typeshift(unit, "Unit")
# bind is unused in this demo, but can be added in principle
system.add_binary_rule(bind, "Bind")
system.add_binary_rule(apply, "LiftedFA")
# do a bit of cleanup, we don't need PM here.
system.remove_rule('PM')
system.typeshift = True
system

**Observation**: `bind` can be used to define a composition rule, but I have not found a lot of uses for the reader monad's specific `bind` in terms of semantic composition, beyond lifting `Apply`.

* There are other monads where the `bind` has a non-trivial direct use, see e.g. Charlow's scopal pied piping.
* A candidate case: if unary predicates are type $\langle e, \langle s, t \rangle \rangle$, then they can compose with a type $e$ subject via `Unit` (on the subject) + `Bind`. However, this is accidental it seems -- doesn't generalize to $n$-ary predicates!

**Third**: define some lexical entries using the appropriate types.

* extensional entries are allowed
* general principle: if an entry accesses a world of evaluation (and isn't a shifter), the $\lambda w$ should be out front. (Aka: intensionalizations of lexical entries that would have ordinary type $\alpha$ are always $\mathbb{M}\alpha$.)
* A shifter (as in the vF&H system) should take an $\mathbb{M}$ type as an argument directly, and return an $\mathbb{M}$ type.

In [ ]:
%%lamb
# non-intensional entries
||Alfonso|| = Alfonso_e
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : f(x) >> g(x)
||and_|| = L p_t : L q_t : p & q
# entries that access a world
||cat|| = L w_s : L x_e : Cat(w,x)
||dance|| = L w_s : L x_e : Dance(w,x)
# shifter example. Note that the world argument goes *inside*...
||might|| = L p_<s,t> : L w_s : Exists w2_s : ((w2 << Epi_<s,{s}>(w)) & p(w2))


Basic case, no shifting:

    (1) Alfonso is a cat.

(I'll supprress the vacuous elements here to keep things simple.)

In [ ]:
(Alfonso * cat).tree()

Add in a shifter:
    
    (2) Alfonso might dance.

n.b., we assume the following LF:

    (2') [might [Alfonso dance]]

In [ ]:
(might * (Alfonso * dance)).tree()

What happens with higher-order functions? What happens when we embed shifting under non-shifting?

    (3) Alfonso is a cat and Alfonso might dance.

In [ ]:
((Alfonso * cat) * (and_ * (might * (Alfonso * dance)))).tree()

Another higher-order function example:

    (4) Every cat dances.

In [ ]:
(every * cat).tree()

In [ ]:
((every * cat) * dance).tree()

    (5) Every cat might dance.
    (5') [might [[every cat] dance]]

**Generalization**: Higher-order non-intensional functions combine via `Unit` + repeated applications of `LiftedFA`.

In [ ]:
(might * ((every * cat) * dance)).tree()

Note: we may or may not get this but there are non-trivial empirical and theoretical questions about scoping in examples like this...what about the other scope? It is not easy to do with the standard PA rule. (Alex's first project!)

* exercise for reader: PA rule that produces a reader $\mathbb{M}$ type...this is entirely possible!

In [ ]:
(((lang.Binder(5) * (might *(lang.Trace(5) * dance))))).tree()

In [ ]:
(every * cat)

In [ ]:
# this fails...
(every * cat) * ((lang.Binder(5) * (might *(lang.Trace(5) * dance))))

**A note on lifting to monadic types**. In principle one can define n-ary lift options that are analogues of `Unit`, for example:

In [ ]:
and_.content

In [ ]:
lift2 = %te L f_<X,<Y,Z>> : L w_s : L x_<s,X> : L y_<s,Y> : f(x(w))(y(w))
lift2

In [ ]:
lifted_and = lift2(and_.content).reduce_all()
lifted_and

In [ ]:
%lamb ||and2|| = (λ w_s: (λ x_<s,t>: (λ y_<s,t>: (x_<s,t>(w_s) & y_<s,t>(w_s)))))

However, `LiftedFA` is still need to compose the result! So this isn't necessary in the present system.

In [ ]:
((Alfonso * cat) * (and2 * (might * (Alfonso * dance)))).tree()